In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [2]:
X_train= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/X_train')
y_train= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/y_train')
X_val= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/X_val')
y_val= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/y_val')
X_test= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/X_test')
y_test= pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Final Data/y_test')

In [3]:
from sklearn.model_selection import RandomizedSearchCV

In [4]:
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 400, 800]}
xgbr = xgb.XGBRegressor(seed = 42)
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=2)

In [5]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END colsample_bylevel=0.6, colsample_bytree=0.4, learning_rate=0.01, max_depth=15, n_estimators=100, subsample=0.5; total time=   8.5s
[CV] END colsample_bylevel=0.6, colsample_bytree=0.4, learning_rate=0.01, max_depth=15, n_estimators=100, subsample=0.5; total time=   7.8s
[CV] END colsample_bylevel=0.6, colsample_bytree=0.4, learning_rate=0.01, max_depth=15, n_estimators=100, subsample=0.5; total time=   7.4s
[CV] END colsample_bylevel=0.6, colsample_bytree=0.4, learning_rate=0.01, max_depth=15, n_estimators=100, subsample=0.5; total time=   8.8s
[CV] END colsample_bylevel=0.6, colsample_bytree=0.4, learning_rate=0.01, max_depth=15, n_estimators=100, subsample=0.5; total time=  14.7s
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.7999999999999999, learning_rate=0.01, max_depth=3, n_estimators=800, subsample=0.6; total time=  48.7s
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.799

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,...
                                          predictor=None, random_state=None, ...),
                   n_iter=25,
                   param_distributions={'colsample_bylevel': array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'colsample_bytree': array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'learning_rate': [0.01, 0.1, 0.2, 0.3],
                                        'max_depth': [3, 5, 6, 10, 15, 20],
                                        'n_estimators': [100, 400, 800],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
                   scoring='neg_mean_squared_error', verbose=2)

In [6]:
clf.best_params_

{'subsample': 0.6,
 'n_estimators': 800,
 'max_depth': 3,
 'learning_rate': 0.01,
 'colsample_bytree': 0.7999999999999999,
 'colsample_bylevel': 0.8999999999999999}

In [7]:
best_params = {'subsample': 0.6,
 'n_estimators': 800,
 'max_depth': 3,
 'learning_rate': 0.01,
 'colsample_bytree': 0.7999999999999999,
 'colsample_bylevel': 0.8999999999999999}

In [10]:
best_xgb = xgb.XGBRegressor(**best_params)
best_xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.8999999999999999, colsample_bynode=None,
             colsample_bytree=0.7999999999999999, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=800, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [11]:
train_preds = best_xgb.predict(X_train)

In [12]:
val_preds = best_xgb.predict(X_val)

In [13]:
train_rf_mae = mean_absolute_error(y_train,train_preds)
train_rf_rmse = np.sqrt(mean_squared_error(y_train,train_preds))

print(train_rf_mae,train_rf_rmse)

20.047570908970393 35.009970128854214


In [14]:
train_rf_mae = mean_absolute_error(y_val,val_preds)
train_rf_rmse = np.sqrt(mean_squared_error(y_val,val_preds))

print(train_rf_mae,train_rf_rmse)

15.254687113812523 29.636863868625593
